In [244]:
import sys
import soundcloud
import networkx as nx

In [245]:
from functools import partial
from utils import get_results, handle_http_errors

In [246]:
def getAttr(resource, attr):
    if hasattr(resource, attr): return getattr(resource, attr)
    return None

getUsername = partial(getAttr, attr='username')
getid = partial(getAttr, attr='id')
getUserid = partial(getAttr, attr='user_id')

In [247]:
def profile2id(profile):
    print("get user id result is {}".format(getUserid(profile)))
    print("get id result is {}".format(getid(profile)))
    return getUserid(profile) if isinstance(getUserid(profile), int) else getid(profile)

In [248]:
def profile2username(profile):
    return getUsername(profile) if isinstance(getUsername(profile), str) else str(getid(profile))

In [249]:
# A global profile graph used to iterate through the various algorithms.
# Each node is profile id, with edges weighted by activity between then.
profile_graph = nx.MultiDiGraph()


In [250]:

client = soundcloud.Client(client_id='454aeaee30d3533d6d8f448556b50f23')
name = input("Enter a soundcloud artist to analyze: ")

Enter a soundcloud artist to analyze: Sybyr


In [251]:
# Artist of interest

search = client.get('/users/', q = name)[0]

In [252]:
search.__dict__

{'obj': {'avatar_url': 'https://i1.sndcdn.com/avatars-jDucfE5tYobXzMC1-o76g6g-large.jpg',
  'id': 13377746,
  'kind': 'user',
  'permalink_url': 'https://soundcloud.com/sybyr',
  'uri': 'https://api.soundcloud.com/users/13377746',
  'username': 'Sybyr (ProdByLandfill)',
  'permalink': 'sybyr',
  'last_modified': '2020/11/22 13:23:27 +0000',
  'first_name': 'ProdbyLandfill.com',
  'last_name': '',
  'full_name': 'ProdbyLandfill.com',
  'city': '',
  'description': 'bookings/email: sybyrcentral@gmail.com\nAnti-World: http://antiworldwide.co (@antiworldglobe) on all networks\n(Prod. by Landfill) at http://prodbylandfill.com',
  'country': 'United States',
  'track_count': 442,
  'public_favorites_count': 0,
  'reposts_count': 110,
  'followers_count': 53001,
  'followings_count': 24,
  'plan': 'Pro Unlimited',
  'myspace_name': None,
  'discogs_name': None,
  'website_title': 'click Here',
  'website': 'http://antiworld.lnk.to/SYBYR',
  'comments_count': 110,
  'online': False,
  'likes_c

In [253]:
print("Artist interpreted as: %s" % getUsername(search))

Artist interpreted as: Sybyr (ProdByLandfill)


In [254]:
uid2username_cache = {}
tid2username_cache = {}
cid2username_cache = {}
tid2userid_cache = {}
cid2userid_cache = {}

# Creating Decorator: handle_http_errors

In [255]:
@handle_http_errors
def userid2username(uid):
    global uid2username_dict
    username = uid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    uid_str = str(uid)
    get = '/users/{}'.format(uid_str)
    result = client.get(get)
    username = result.username
    uid2username_cache[uid] = username
    return username

@handle_http_errors
def trackid2username(tid):
    global tid2username_dict
    username = tid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    tid_str = str(tid)
    get = '/tracks/{}'.format(tid_str)
    result = client.get(get)
    username = result.user['username']
    tid2username_cache[tid] = username
    return username

@handle_http_errors
def commentid2username(cid):
    global cid2username_dict
    username = cid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    cid_str = str(cid)
    get = '/comments/{}'.format(cid_str)
    result = client.get(get)
    username = result.user['username']
    cid2username_cache[cid] = username
    return username

@handle_http_errors
def trackid2userid(tid):
    global tid2userid_dict
    userid = tid2userid_cache.get(tid, None)
    if userid is not None: 
        return userid
    # username is none, we don't have it in cache
    
    tid_str = str(tid)
    get = '/tracks/{}'.format(tid_str)
    result = client.get(get)
    userid = result.user['id']
    tid2userid_cache[tid] = userid
    return userid

@handle_http_errors
def commentid2userid(cid):
    global cid2userid_dict
    username = cid2username_cache.get(cid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    cid_str = str(cid)
    get = '/comments/{}'.format(cid_str)
    result = client.get(get)
    userid = result.user['id']
    cid2userid_cache[cid] = userid
    return userid
    

# get social data functions

In [256]:
def getRelationships(profile, client, url): 
    return get_results(client, url)

@handle_http_errors
def getFollows(profile):
    # get all profiles a user follows
    follows = get_results(client, '/users/{0:s}/followings/'.format(str(profile)))
    return follows

@handle_http_errors
def getFollowers(profile):
    # get all profiles that follow a user
    followers = get_results(client, '/users/{0:s}/followers/'.format(str(profile)))
    return followers

@handle_http_errors
def getUserFavorites(profile):
    user_favorites = get_results(client, '/users/{0:s}/favorites/'.format(str(profile)))
    return user_favorites

@handle_http_errors
def getUserComments(profile):
    comments = get_results(client, '/users/{0:s}/comments/'.format(str(profile)))
    return comments

@handle_http_errors
def getTracks(profile):
    tracks = get_results(client, '/users/{0:s}/tracks/'.format(str(profile)))
    return tracks

@handle_http_errors
def getTrackComments(track):
    track_comments = get_results(client, '/tracks/{0:s}/comments/'.format(str(track)))
    return track_comments

@handle_http_errors
def getTrackFavoriters(track):
    track_favoriters = get_results(client, '/tracks/{0:s}/favoriters/'.format(str(track)))
    return track_favoriters

# build graph functions

In [257]:
def addProfileNx(profile, profileGraph):
    profileGraph.add_node(profile)
    #return profileGraph

def addPairNx(profile, neighbor, profileGraph):
    addProfileNx(profile)
    addProfileNx(neighbor)
    
def addFollowNx(profile, follow, profileGraph):
    # enforce that already existing follows don't get added again
    if not profileGraph.has_edge(profile, follow, 'soundcloud_follow'):
        profileGraph.add_edge(profile, follow, 'soundcloud_follow',weight=1)

def addFollowsNx(profile, follows, profileGraph):
    for follow in follows:
        addFollowNx(profile, follow, profileGraph)

def addFollowersNx(profile, followers, profileGraph):
    for follower in followers:
        addFollowNx(follower, profile, profileGraph)

# TODO: enforce that the same favorite isn't counted twice
# easy solve: keep a list of tracks as an edge attribute
def addFavoriteNx(profile, favorite, track, profileGraph):
    # if we already saw a favorite from this user before...
    if profileGraph.has_edge(profile, favorite, 'soundcloud_favorite'):
        # but not for this track
        if track not in profileGraph.edges[profile, favorite, 'soundcloud_favorite']['tracks']:
            profileGraph.edges[profile, favorite, 'soundcloud_favorite']['tracks'] = profileGraph.edges[profile, favorite, 'soundcloud_favorite'].get('tracks', []) + [track]
    else:
        # we never saw this favorite from this profile before, so we need to create a new edge "type" or key
        profileGraph.add_edge(profile, favorite, 'soundcloud_favorite')#,weight=1)
        profileGraph.edges[profile, favorite, 'soundcloud_favorite']['tracks'] = [track]
    profileGraph.edges[profile, favorite,'soundcloud_favorite']['weight'] = len(profileGraph.edges[profile, favorite, 'soundcloud_favorite']['tracks'])        


def addUserFavoritesNx(profile, user_favorites, profileGraph):
    for track, user_favorite in user_favorites:
        addFavoriteNx(profile, user_favorite, track, profileGraph)
        # if we already saw a favorite from this user before...
#         if profileGraph.has_edge(profile, user_favorite, 'soundcloud_favorite'):
#             # but not for this track
#             if track not in profileGraph.edges[profile, user_favorite, 'soundcloud_favorite']['tracks']:
#                 profileGraph.edges[profile, user_favorite, 'soundcloud_favorite']['tracks'] = profileGraph.edges[profile, user_favorite, 'soundcloud_favorite'].get('tracks', []) + [track]
#         else:
#             # we never saw a favorite before, so need to create a new edge "type"
#             profileGraph.add_edge(profile, user_favorite, 'soundcloud_favorite')#,weight=1)
#             profileGraph.edges[profile, user_favorite, 'soundcloud_favorite']['tracks'] = [track]
#         profileGraph.edges[profile, user_favorite,'soundcloud_favorite']['weight'] = len(profileGraph.edges[profile, user_favorite, 'soundcloud_favorite']['tracks'])        
            
def addTrackFavoriters(profile, track_favoriters, profileGraph):
    for track, track_favoriter in track_favoriters:
        addFavoriteNx(track_favoriter, profile, track, profileGraph)
#         if profileGraph.has_edge(track_favoriter, profile, 'soundcloud_favorite'):
#             profileGraph.edges[track_favoriter, profile,'soundcloud_favorite']['weight']+=1
#         else:
#             profileGraph.add_edge(track_favoriter, profile, 'soundcloud_comment',weight=1)

# TODO: enforce that the same comment isn't counted twice
# easy solve: keep a dictionary of track-comments pairs as an edge attribute
def addUserCommentsNx(profile, user_comments, profileGraph):
    for user_comment in user_comments:
        if profileGraph.has_edge(profile, user_comment, 'soundcloud_comment'):
            profileGraph.edges[profile, user_comment,'soundcloud_comment']['weight']+=1
        else:
            profileGraph.add_edge(profile, user_comment, 'soundcloud_comment',weight=1)
            
def addTrackCommenters(profile, track_commenters, profileGraph):
    for track_commenter in track_commenters:
        if profileGraph.has_edge(track_commenter, profile, 'soundcloud_comment'):
            profileGraph.edges[track_commenter, profile,'soundcloud_comment']['weight']+=1
        else:
            profileGraph.add_edge(track_commenter, profile, 'soundcloud_comment',weight=1)
            

# print graph functions

In [258]:
def printGraph(G):
    for profile, neighbor, key, weight in G.edges(data=True,keys=True):
        print("Profile: {}".format(userid2username(profile)))
        print("Neighbor: {}".format(userid2username(neighbor)))
        print(key)
        print(weight)
#     for profile in G.nodes():
#         if profile:
#             try:
#                 profile_name = id2username(profile)
#                 print("\t", "Printing: %s (%s)" % (profile_name, profile))
#                 # why does successors return an iterator instead of a list? getting fancy for no reason
#                 follows = [follow for follow in G.successors(profile)]
#                 followers = [follower for follower in G.predecessors(profile)]
#                 try:    
#                     print("\t", profile_name + " has " + str(len(follows)) + " followings")
#                     print("\t", profile_name + " follows " + ", ".join(map(lambda x: id2username(x), follows)))
#                 except TypeError: 
#                     print("No follows here!")
#                 try:    
#                     print("\t", profile_name + " has " + str(len(followers)) + " followers")
#                     print("\t", profile_name + " is followed by " + ", ".join(map(lambda x: id2username(x), followers)))
#                 except TypeError:
#                     print("No followers here!")
#                     print("-"*40)
#             except UnicodeError:
#                 print("Artist's username not found" )   

# 1b. Testing Functions

In [259]:
# list of profiles to query
profiles_to_query = [search.id]
profiles_to_query

[13377746]

In [260]:
depth = 1
i = 0

# list of profiles we could not query
unavailable_profiles = []

Only uncomment the code in the loop if you are willing to wait a while to run the cell below:

In [261]:
for t in range(depth):
    print("Iteration " + str(t))
    profiles_to_query = list(set(profiles_to_query))
    
    for profile in profiles_to_query:
        username = userid2username(profile)
        if username:
            print("\t", "Enqueueing: %s (%s)" % (username, profile))
            addProfileNx(profile,profile_graph)

#             encountered_follows = getFollows(profile)
#             encountered_follow_profiles = [follow.id for follow in encountered_follows]
#             print("encountered follows: " + ", ".join([userid2username(profile) for profile in encountered_follow_profiles]))
#             print('\n')
#             addFollowsNx(profile, encountered_follow_profiles, profile_graph)

#             encountered_user_favorites = getUserFavorites(profile)
#             encountered_user_favorite_tracks_profiles = [(track.title, getUserid(track) if isinstance(getUserid(track), int) else track.user_id) for track in encountered_user_favorites]
#             print("encountered user favorite profiles: " + ", ".join([userid2username(profile) for track_title, profile in encountered_user_favorite_tracks_profiles]))
#             print('\n')
#             addUserFavoritesNx(profile, encountered_user_favorite_tracks_profiles, profile_graph)

#             encountered_user_comments = getUserComments(profile)
#             encountered_user_comment_profiles = [trackid2userid(comment.track_id) for comment in encountered_user_comments]
#             print("encountered user comment profiles: " + ", ".join([userid2username(profile) for profile in encountered_user_comment_profiles]))
#             addUserCommentsNx(profile, encountered_user_comment_profiles, profile_graph)
#             print('\n')

#             encountered_followers = getFollowers(profile)
#             encountered_follower_profiles = [follower.id for follower in encountered_followers]
#             print("encountered followers: " + ", ".join([userid2username(profile) for profile in encountered_follower_profiles]))
#             print('\n')
#             addFollowersNx(profile, encountered_follower_profiles, profile_graph)

            encountered_tracks = getTracks(profile)

            encountered_track_comments = []
            encountered_comment_tracks = []
            encountered_track_favoriters = []
            encountered_favoriter_tracks = []

            for track in encountered_tracks:
                track_title = track.title
                track_id = track.id
                encountered_track_comments_len_before = len(encountered_track_comments)
                encountered_track_favoriters_len_before = len(encountered_track_favoriters)
                print("getting comments / favoriters from user track, {}".format(track_title))
                encountered_track_comments += getTrackComments(track_id)
                encountered_comment_tracks = (len(encountered_track_comments) - encountered_track_comments_len_before)*track_title
                encountered_track_favoriters += getTrackFavoriters(track_id)
                encountered_favoriter_tracks = (len(encountered_track_favoriters) - encountered_track_favoriters_len_before)*track_title
            
            encountered_track_comments = zip(encountered_comment_tracks, encountered_track_comments)
            encountered_track_favoriters = zip(encountered_favoriter_tracks, encountered_track_favoriters)
            

#             print(encountered_track_comments[0].__dict__)
#             encountered_track_commenters = [(comment.user['id'], comment.user['username']) for comment in encountered_track_comments]
#             encountered_track_commenter_ids = [encountered_track_commenter[0] for encountered_track_commenter in encountered_track_commenters]
#             encountered_track_commenter_usernames = [encountered_track_commenter[1] for encountered_track_commenter in encountered_track_commenters]
#             print("encountered track comments: " + ", ".join(encountered_track_commenter_usernames))
#             print('\n')
#             addTrackCommenters(profile, encountered_track_commenter_ids, profile_graph)
            
            # need to update code below to initially store results in a list, as above
            # grab all people who have favorited a track of the target artist
            print(encountered_track_favoriters)
            print(encountered_track_favoriters[0])
            print(encountered_track_favoriters[0][1])#.__dict__)
            encountered_track_favoriter_ids = [(track_title, encountered_track_favoriter.id) for track_title, encountered_track_favoriter in encountered_track_favoriters]
            #print(encountered_track_favoriter_ids)
            encountered_track_favoriter_usernames = [userid2username(encountered_track_favoriter_id) for track_title, encountered_track_favoriter_id in encountered_track_favoriter_ids]
            print(encountered_track_favoriter_usernames)
            print("encountered track favoriters: " + ", ".join([encountered_track_favoriter_username if isinstance(encountered_track_favoriter_username, str) else " ".join(encountered_track_favoriter_username) for encountered_track_favoriter_username in encountered_track_favoriter_usernames]))
            addTrackFavoriters(profile, encountered_track_favoriter_ids, profile_graph)
            
        else:
            print("\t", "Artist ID %s is not query-able" % profile)
            unavailable_profiles.append(profile)            

Iteration 0
	 Enqueueing: Sybyr (ProdByLandfill) (13377746)
getting comments / favoriters from user track, Playboi Carti - Magnolia (Sybyr Remix)
getting comments / favoriters from user track, Playboi Carti - Foreign Sybyr Remix
getting comments / favoriters from user track, Playboi Carti - Neon (Sybyr Remix)
getting comments / favoriters from user track, Hey You, Remix This (Despicable Acapella)
getting comments / favoriters from user track, YOU WANT TO BE SYBYR BUT NOT GIVE SYBYR HIS DUE PLACE. (Prod. Shark)
getting comments / favoriters from user track, How I Feel About Now (Play For The Band)
getting comments / favoriters from user track, ProdbyLandfill.com (feat. Landfill)
getting comments / favoriters from user track, Do It Again. Prod Kiddbeatz
getting comments / favoriters from user track, Back Prod Babe
getting comments / favoriters from user track, Let The Church Say Bang ProdbyLandfill.com
getting comments / favoriters from user track, Level - Headed Prod Kiddbeatz
getting c

getting comments / favoriters from user track, [a Thunderstorm Suddenly Appeared] (PROD. @LORDLANDFILL)
getting comments / favoriters from user track, Full Stream - Sybyrmusic.com
getting comments / favoriters from user track, WHAT DO YOU SERIOUSLY WANT? (NEW EP ON BANDCAMP JULY 21ST)
getting comments / favoriters from user track, ProdbyLandfill.com ^leak$ '19 / 9696 - yingyang
getting comments / favoriters from user track, Gotta Serve Folks (Prod. BruhManeGod)
getting comments / favoriters from user track, Hate My New Shit (Prod. Shark)
getting comments / favoriters from user track, I Said I'm Done Playing (Prod. Landfill)
getting comments / favoriters from user track, Needa Shut The Fuck Up (Prod. Landfill)
getting comments / favoriters from user track, Neurotic [No I Can't Relax] (Prod. Throwedtobin)
getting comments / favoriters from user track, Some Haters Then [Needleheads] (Prod. Kiddbeatz)
getting comments / favoriters from user track, Suffocate (Prod. Shark)
getting comments /

getting comments / favoriters from user track, Sy
getting comments / favoriters from user track, I Want Tu Be A Coon
getting comments / favoriters from user track, I Found My Way
getting comments / favoriters from user track, Rolex On My Dick
getting comments / favoriters from user track, If My Whip Takes Discs
getting comments / favoriters from user track, I Will Grab The Belt
getting comments / favoriters from user track, How Does It Feel? (Hi Hater)
getting comments / favoriters from user track, Potatoes Or Woodchips
getting comments / favoriters from user track, 30 Aint 30
getting comments / favoriters from user track, 3830
getting comments / favoriters from user track, Ride My Ballsack
getting comments / favoriters from user track, It Is A Waste
getting comments / favoriters from user track, Heart Attack, Gimme That
getting comments / favoriters from user track, Welp (ft. Eric North)
getting comments / favoriters from user track, Take Ya Shine.wav
getting comments / favoriters fro

HTTPError: 503 Server Error: Service Unavailable for url: https://api.soundcloud.com/tracks/570890703/favoriters?client_id=454aeaee30d3533d6d8f448556b50f23&cursor=1567440371596422&linked_partitioning=1&order=created_at&page_size=100&client_id=454aeaee30d3533d6d8f448556b50f23

In [74]:
printGraph(profile_graph)

Profile: Sybyr (ProdByLandfill)
Neighbor: LIL MAI 2021
soundcloud_favorite
{'tracks': ["We're Dying (Prod. Retro Black)", 'What Is Sane? prod. Landfill', '❤ STFU ❤(Sped Up)  prod candymane', 'going back 2 Indiuh 💯(PROD. LOFTY305)', 'Us Alone - SYBYR x LIL MAI (Prod. laura les)'], 'weight': 5}
Profile: Sybyr (ProdByLandfill)
Neighbor: chloehotline
soundcloud_favorite
{'tracks': ['RANGE (w/ Malik!)'], 'weight': 1}
Profile: Sybyr (ProdByLandfill)
Neighbor: naj
soundcloud_favorite
{'tracks': ['youknowit'], 'weight': 1}
Profile: Sybyr (ProdByLandfill)
Neighbor: Lean Quatifah
soundcloud_favorite
{'tracks': ['Yas Indeed'], 'weight': 1}
Profile: Sybyr (ProdByLandfill)
Neighbor: LAKIM
soundcloud_favorite
{'tracks': ['LAKIM w | McClenney - Live From The Dorm (UMD)'], 'weight': 1}
Profile: Sybyr (ProdByLandfill)
Neighbor: TEK.LUN
soundcloud_favorite
{'tracks': ['Sometimes (Alldatjazz)'], 'weight': 1}
Profile: Sybyr (ProdByLandfill)
Neighbor: PLANET GIZA
soundcloud_favorite
{'tracks': ['Funky Komp

# 2. metadata functions

In [75]:
def getUserInfo(profile):
    info = ['id','permalink', 'username', 'avatar_url', 'country', 'city', 
            'website', 'track_count', 'followers_count', 'followings_count']
    return {i: getAttr(profile, i) for i in info}

def getTrackInfo(track):
    info = ['id', 'user_id', 'created_at', 'streamabale', 'downloadable',
            'playback_count', 'download_count', 'favoritings_count','comment_count']
    return {i: getAttr(track, i) for i in info}

def getUserFavInfo(fav):
    info = ['id', 'user_id','last_modified']
    return {i: getAttr(fav, i) for i in info}

def getTrackFavInfo(fav):
    info = ['id', 'username', 'last_modified']
    return {i: getAttr(fav, i) for i in info}

def getUserCommInfo(comm):
    info = ['id', 'user_id', 'track_id', 'timestamp']
    return {i: getAttr(comm, i) for i in info}

def getTrackCommInfo(comm):
    info = ['id', 'user_id', 'track_id','timestamp']
    return {i: getAttr(comm, i) for i in info}

# 3. dump graph

In [77]:
#nx.write_graphml(profile_graph, 'profile.graphml')

In [78]:
print("Done.")

Done.
